In [ ]:
#!pip install datasets
#!pip install SPARQLWrapper

In [49]:
from datasets import load_dataset
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import json


### Extração do dataset

In [ ]:

# extrair o dataset
# dataset = load_dataset("manoh2f2/tsterbak-lyrics-dataset-with-emotions")

# # print(dataset)                 # Mostra as divisões (train, test, etc.)
# # print(dataset['train'][0])     # Mostra o primeiro 


# df = dataset['train'].to_pandas()
# # print(df.head())
# x = df.iloc[2]['seq']
# print(x)

df = pd.read_parquet("hf://datasets/manoh2f2/tsterbak-lyrics-dataset-with-emotions/data/train-00000-of-00001.parquet")

# print(df)

#x = df.iloc[2]['seq']
#print(x)


In [ ]:
cache = {}

### Query do SPARQL

In [57]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
cache_file = "genre_cache.json"
# cache_str_keys é do tipo { "song||artist": genre }
with open("genre_cache.json", "r", encoding="utf-8") as f:
    cache_str_keys = json.load(f)


def get_genre2(artist_name):
    query = f"""
    SELECT ?genreLabel WHERE {{
      ?artist rdfs:label "{artist_name}"@en;
              wdt:P136 ?genre.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if results["results"]["bindings"]:
        return results["results"]["bindings"][0]["genreLabel"]["value"]

    else:
        return "Unknown"


def get_genre(song_name,artist_name):
    
    song_name = song_name.replace('"', '').replace("'", '')
    artist_name = artist_name.replace('"', '').replace("'", '')
    query = f"""
    SELECT ?genreLabel WHERE {{
      ?artist rdfs:label "{song_name}"@en;
              wdt:P136 ?genre.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    LIMIT 1
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    if results["results"]["bindings"]:
        return results["results"]["bindings"][0]["genreLabel"]["value"]

    else:
        return get_genre2(artist_name) # Caso não encontre género para a música em especifico, vai buscar o género do artista



def get_genre_cached(song_name, artist_name):
    key = f"{song_name}||{artist_name}"  # chave como string
    if key in cache_str_keys:
        print("skippou")
        return cache_str_keys[key]
    
    genre = get_genre(song_name, artist_name)  # faz SPARQL se não estiver
    cache_str_keys[key] = genre

    # salva a cache imediatamente no JSON
    with open("genre_cache.json", "w", encoding="utf-8") as f:
        json.dump(cache_str_keys, f, ensure_ascii=False, indent=2)

    return genre



artist = df.iloc[2]["artist"]
song = df.iloc[2]["song"]
genre = get_genre(song,artist)

print(f"Artista: {artist}")
print(f"Género: {genre}")

#df["genre"] = df["artist"].apply(get_genre)

df["genre"] = [get_genre_cached(title, artist) for title, artist in zip(df["song"], df["artist"])]


df.to_csv("lyrics_with_genre.csv", index=False)



Artista: Elijah Blake
Género: contemporary R&B
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
skippou
s

KeyboardInterrupt: 

In [ ]:
print(cache)

# cache_str_keys = {f"{song}||{artist}": genre for (song, artist), genre in cache.items()}
# # Salvar cache atual
# with open(cache_file, "w", encoding="utf-8") as f:
#     json.dump(cache_str_keys, f, ensure_ascii=False, indent=2)


print(df[df["song"] == "Love in Space"].index)
print(cache_str_keys)


{('Everyday', 'Elijah Blake'): 'pop music', ('Live Till We Die', 'Elijah Blake'): 'contemporary R&B', ('Pinot', 'Elijah Blake'): 'contemporary R&B', ('Uno', 'Elijah Blake'): 'alternative rock', ('Abendlied', 'Elis'): 'motet', ('Child', 'Elis'): 'pop music', ('Come to Me', 'Elis'): 'hip-hop', ('Engel der Nacht', 'Elis'): 'gothic metal', ('My Only Love', 'Elis'): 'romance film', ('Sie Erfasst Mein Herz', 'Elis'): 'gothic metal', ('Anger', 'Elis'): 'allegory', ('Black Angel', 'Elis'): 'drama film', ('Devil Inside You', 'Elis'): 'gothic metal', ('Heart in Chains', 'Elis'): 'gothic metal', ('Lost Soul', 'Elis'): 'technical death metal', ('Perfect Love', 'Elis'): 'country music', ('Rebirth', 'Elis'): 'contemporary R&B', ('The Burning', 'Elis'): 'horror film', ('Forgotten Love', 'Elis'): 'traditional folk music', ('Phoenix from the Ashes', 'Elis'): 'gothic metal', ('Remember the Promise', 'Elis'): 'gothic metal', ('Show Me the Way', 'Elis'): 'pop music', ('I Come Undone', 'Elis'): 'gothic met